# Generalized Higher-Order Orthogonal Iteration for Tensor Decomposition and Completion

This notebook shows how to implement a gHOI imputer on three real-world data sets (i.e., PeMS traffic speed data, Guangzhou traffic speed data, Electricity data). To overcome the problem of missing values within multivariate time series data, this method takes into account both tensor tucker decomposition and low rank core tensor structure. For an in-depth discussion of gHOI, please see [1].

<div class="alert alert-block alert-info">
<font color="black">
<b>[1]</b> Yuanyuan Liu, Fanhua Shang, Wei Fan, James Cheng,  Hong Cheng (2014). <b>Generalized Higher-Order Orthogonal Iteration for Tensor Decomposition and Completion</b>. NIPS Proceedings. <a href="https://papers.nips.cc/paper/5476-generalized-higher-order-orthogonal-iteration-for-tensor-decomposition-and-completion.pdf" title="PDF"><b>[PDF]</b></a> 
</font>
</div>


In [1]:
import numpy as np
import time
from numpy.linalg import inv as inv

### Define LATC-imputer kernel

We start by introducing some necessary functions that relies on `Numpy`.

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>ten2mat</code>:</b> <font color="black">Unfold tensor as matrix by specifying mode.</font></li>
<li><b><code>mat2ten</code>:</b> <font color="black">Fold matrix as tensor by specifying dimension (i.e, tensor size) and mode.</font></li>
<li><b><code>tucker_combine</code>:</b> <font color="black">Combine core tensor and unitary matrices as full tensor.</font></li>
</ul>
</div>

### Tensor folding and unfolding

In [2]:
def ten2mat(tensor, mode):
    return np.reshape(np.moveaxis(tensor, mode, 0), (tensor.shape[mode], -1), order = 'F')

def mat2ten(mat, dim, mode):
    index = list()
    index.append(mode)
    for i in range(dim.shape[0]):
        if i != mode:
            index.append(i)
    return np.moveaxis(np.reshape(mat, list(dim[index]), order = 'F'), 0, mode)

## Tensor tucker combination

In [3]:
def tucker_combine(var, skip = False, skipn = 0):
    annotation = 'qwertyuiop'
    G = var[0]
    R = G.shape
    dim_N = len(R)
    anno = annotation[:dim_N]
    W = G.copy()
    for n in range(len(var) - 1):
        if skip == True and n == skipn:
            continue
        target = anno.replace(anno[n], 'n')
        mul_type = anno + ', n' + annotation[n] + '->' + target
        W = np.einsum(mul_type, W, var[n + 1])
    return W

## Function to randomly initiate variables

In [4]:
def init_variables(dim, R):
    G = np.random.rand(*R)
    dim_N = len(dim)
    U = []
    for i in range(dim_N):
        U.append(np.random.rand(dim[i], R[i]))
        
    V = []
    for i in range(dim_N):
        V.append(ten2mat(G, i))
    
    Y = []
    for i in range(dim_N):
        Y.append(np.zeros_like(V[i]))
    return G, U, V, Y

## Loss Calculator

In [5]:
def losscal(V, Y, G, U, X, X_pre, mu, lambda_l):
    loss = 0
    dim_N = len(X.shape)
    for i in range(dim_N):
        u, s, v = np.linalg.svd(V[i], full_matrices=0)
        mat = ten2mat(G, i) - V[i]
        loss += np.sum(s) + np.einsum('ij, ij', Y[i], mat) + mu / 2 * np.sum(np.square(mat))
    loss += lambda_l * np.sum(np.square(X - X_pre))
    return loss

## Error calculator
<div class="alert alert-block alert-warning">
<ul>
<li><b><code>compute_mape</code>:</b> <font color="black">Compute the value of Mean Absolute Percentage Error (MAPE).</font></li>
<li><b><code>compute_rmse</code>:</b> <font color="black">Compute the value of Root Mean Square Error (RMSE).</font></li>
</ul>
</div>

> Note that $$\mathrm{MAPE}=\frac{1}{n} \sum_{i=1}^{n} \frac{\left|y_{i}-\hat{y}_{i}\right|}{y_{i}} \times 100, \quad\mathrm{RMSE}=\sqrt{\frac{1}{n} \sum_{i=1}^{n}\left(y_{i}-\hat{y}_{i}\right)^{2}},$$ where $n$ is the total number of estimated values, and $y_i$ and $\hat{y}_i$ are the actual value and its estimation, respectively.

In [6]:
def compute_mape(var, var_hat):
    return np.sum(np.abs(var - var_hat) / var) / var.shape[0]

def compute_rmse(var, var_hat):
    return  np.sqrt(np.sum((var - var_hat) ** 2) / var.shape[0])

## Function validity test
<div class="alert alert-block alert-warning">
<ul>
<li><b><code>tucker_combination</code>:</b> <font color="black">Test the function validity of `tucker_combination`</font></li>
<li><b><code>padding</code>:</b> <font color="black">Test the validity of tensor padding.</font></li>
</ul>
</div>

### tucker_combination test

In [7]:
G = np.random.rand(1, 2, 3)
U_1 = np.random.rand(3, 1)
U_2 = np.random.rand(2, 2)
U_3 = np.random.rand(1, 3)
var = [G, U_1, U_2, U_3]
tucker_cb = tucker_combine(var)
print(tucker_cb.shape)
dim1 = U_1.shape[0]
dim2 = U_2.shape[0]
dim3 = U_3.shape[0]

R1 = U_1.shape[1]
R2 = U_2.shape[1]
R3 = U_3.shape[1]

GU = np.zeros((dim1, R2, R3))
for i in range(dim1):
    for j in range(R2):
        for k in range(R3):
            GU[i,j,k] = np.matmul(G[:, j, k], U_1[i, :])

GUU = np.zeros((dim1, dim2, R3))
for i in range(dim1):
    for j in range(dim2):
        for k in range(R3):
            GUU[i,j,k] = np.matmul(GU[i, :, k], U_2[j, :])
            
GUUU = np.zeros((dim1, dim2, dim3))
for i in range(dim1):
    for j in range(dim2):
        for k in range(dim3):
            GUUU[i,j,k] = np.matmul(GUU[i, j, :], U_3[k, :])
print('Func: tucker_combine result:')
print(tucker_cb)
print()
print('Ground Truth:')
print(GUUU)
print()

# print(tucker_cb.dtype)
# print(GUUU.dtype)
# print()
# print('Dose tucker_cb equal GUUU?')
# if np.array_equal(GUUU, tucker_cb):
#     print('Yes!')
# else:
#     print('No~')

(3, 2, 1)
Func: tucker_combine result:
[[[1.43081079]
  [0.86125167]]

 [[0.83129486]
  [0.50038349]]

 [[0.02660237]
  [0.01601283]]]

Ground Truth:
[[[1.43081079]
  [0.86125167]]

 [[0.83129486]
  [0.50038349]]

 [[0.02660237]
  [0.01601283]]]



### Padding test

In [8]:
# A = np.zeros((5,5))
B = np.array([[[1,1],[2,2]],[[3,3],[4,4]]])
delta = np.array([2, 1, 0])
N_tuple = ()
for i in range(len(delta)):
    N_tuple += ((0, delta[i]), )
C = np.pad(B, N_tuple, 'constant', constant_values=(0))
# C = B.resize((5, 5))
print(C)

[[[1 1]
  [2 2]
  [0 0]]

 [[3 3]
  [4 4]
  [0 0]]

 [[0 0]
  [0 0]
  [0 0]]

 [[0 0]
  [0 0]
  [0 0]]]


## Generalized Higher-Order Orthogonal Iteration Imputer
The following `imputer` kernel includes some necessary inputs:

<div class="alert alert-block alert-warning">
<ul>
<li><b><code>dense_tensor</code>:</b> <font color="black">This is an input which has the ground truth for validation. If this input is not available, you could use <code>dense_tensor = sparse_tensor.copy()</code> instead.</font></li>
<li><b><code>sparse_tensor</code>:</b> <font color="black">This is a partially observed tensor which has many missing entries.</font></li>
<li><b><code>r</code>:</b> <font color="black">Initial n-rank of the aprroximated tensor, e.g., <code>r = np.array([10, 10, 10])</code>. </font></li>
<li><b><code>R_max</code>:</b> <font color="black">The upper bound of the approximated tensor, e.g., <code>R_max = np.array([80, 80, 80])</code>. </font></li>
<li><b><code>lambda_l</code>:</b> <font color="black">Weight for sum of squared residual error  e.g., <code>lambda_l = 1</code>. </font></li>
<li><b><code>rho</code>:</b> <font color="black">Scalling factor of mu, e.g., <code>epsilon = 1.01</code>. </font></li>
<li><b><code>mu0</code>:</b> <font color="black">Initial learning rate for ADMM, e.g., <code>mu0 = 0.0005</code>. </font></li>
<li><b><code>mu_max</code>:</b> <font color="black">Upper bound of learning rate for ADMM, e.g., <code>mu_max = 0.01</code>. </font></li>
<li><b><code>delta</code>:</b> <font color="black">Rank increasing step lengths, e.g., <code>delta = np.array([5, 5, 5])</code>. </font></li>
<li><b><code>epsilon</code>:</b> <font color="black">Rank increasing criteria, e.g., <code>epsilon = 0.2 </code>. </font></li>
<li><b><code>maxiter</code>:</b> <font color="black">Maximum iteration to stop algorithm, e.g., <code>maxiter = 100 </code>. </font></li>
</ul>
</div>

In [9]:
def imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter):
    X = sparse_tensor.copy()
    mu = mu0
    R = r.copy()
    dim = sparse_tensor.shape
    dim_N = len(dim)
    G, U, V, Y = init_variables(dim, R)
    pos = np.where(sparse_tensor == 0)
    pos_test = np.where((dense_tensor != 0) & (sparse_tensor == 0))
    start_time = time.time()
    L_pre = np.inf
    for iteration in range(maxiter):
        # Update unitary matrices
        for i in range(dim_N):
            parse_list = [X]
            for j in range(dim_N):
                    parse_list.append(U[j].T)
            M = tucker_combine(parse_list, skip = True, skipn = i)
            N = np.zeros(R)
            for j in range(dim_N):
                N = N + mat2ten(V[j]-Y[j]/mu, R, j)
            MN = np.matmul(ten2mat(M, i), ten2mat(N, i).T)
            Ud, sd, Vd = np.linalg.svd(MN, full_matrices=0)
            U[i] = np.matmul(Ud, Vd)
        
        # Update core tensor
        parse_list = [X]
        for j in range(dim_N):
            parse_list.append(U[j].T)
        G = lambda_l / (lambda_l + dim_N * mu) * tucker_combine(parse_list, skip = False)
        for j in range(dim_N):
            G = G + mu / (lambda_l + dim_N * mu) * mat2ten(V[j]-Y[j]/mu, R, j)
        
        # Update auxiliary matrices
        for i in range(dim_N):
            Us, ss, Vs = np.linalg.svd(ten2mat(G, i) + Y[i]/mu, full_matrices=0)
            vec = ss - 1 / mu
            vec[vec <= 0] = 0
            V[i] = np.matmul(np.matmul(Us, np.diag(vec)), Vs)
        
        # Update data tensor (imputation)
        parse_list = [G]
        for i in range(dim_N):
            parse_list.append(U[i])
        X_hat = tucker_combine(parse_list, skip = False)
        X_pre = X.copy()
        X[pos] = X_hat[pos].copy()

        # Update multiplier
        for i in range(dim_N):
            Y[i] = Y[i] + mu * (ten2mat(G, i) - V[i])

#         # Stop criteria
#         GVD = []
#         for i in range(dim_N):
#             GVD.append(np.sum(np.square(ten2mat(G, i) - V[i])))
#         tolerance = max(GVD)
#         if tolerance < tol:
#             break
        
        # Rank increasing
        L = losscal(V, Y, G, U, X, X_pre, mu, lambda_l)
        lcr = np.abs(1 - L/L_pre)
        L_pre = L
        delta_c = delta.copy()
        if lcr <= epsilon:
            for i in range(dim_N):
                delta_c[i] = min(delta[i], R_max[i]-R[i])
                if delta_c[i] != 0:
                    H = np.random.rand(dim[i], delta_c[i])
                    U_hat = np.matmul((np.eye(dim[i]) - np.matmul(U[i], U[i].T)), H)
                    U[i] = np.concatenate((U[i], U_hat), axis=1)

            R_pre = R.copy()
            R = R + delta_c
            delta_tuple = ()
            for i in range(dim_N):
                delta_tuple += ((0, delta_c[i]), )
                
            for i in range(dim_N):
                W_cal = mat2ten(V[i], R_pre, i)
                W_cal_c = np.pad(W_cal, delta_tuple, 'constant', constant_values=(0))
                V[i] = ten2mat(W_cal_c, i)

            for i in range(dim_N):
                W_cal = mat2ten(Y[i], R_pre, i)
                W_cal_c = np.pad(W_cal, delta_tuple, 'constant', constant_values=(0))
                Y[i] = ten2mat(W_cal_c, i)
        
        # Update parameter mu
        mu = max(mu * rho, mu_min)
            
        if (iteration + 1) % 100 == 0:
            print('Iteration: %d, Time cost: %ds'%(iteration + 1, time.time() - start_time))
#             print('Tolerance: {:.6}'.format(tolerance))
            print('MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], X[pos_test])))
            print('RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], X[pos_test])))
            print('Current rank:')
            print(R)
            print()
            start_time = time.time()
            
    print('Total iteration: %d'%(iteration + 1))
#     print('Tolerance: {:.6}'.format(tolerance))
    print('Imputation MAPE: {:.6}'.format(compute_mape(dense_tensor[pos_test], X[pos_test])))
    print('Imputation RMSE: {:.6}'.format(compute_rmse(dense_tensor[pos_test], X[pos_test])))
    print('Current rank:')
    print(R)
    return X

### Guangzhou data

We generate **random missing (RM)** values on Guangzhou traffic speed data set.

In [37]:
np.random.seed(111)

In [24]:
import scipy.io

tensor = scipy.io.loadmat('../../Guangzhou-data-set/tensor.mat')
dense_tensor = tensor['tensor']
random_tensor = scipy.io.loadmat('../../Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

missing_rate = 0.2

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

dense_tensor = np.transpose(dense_tensor, [0, 2, 1])
sparse_tensor = np.transpose(sparse_tensor, [0, 2, 1])
print('Tensor shape:')
print(dense_tensor.shape)

Tensor shape:
(214, 144, 61)


We use `imputer` to fill in the missing entries and measure performance metrics on the ground truth.

In [25]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.05
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 14s
MAPE: 0.11413
RMSE: 4.70144
Current rank:
[13  7  4]

Iteration: 200, Time cost: 26s
MAPE: 0.0871941
RMSE: 3.75304
Current rank:
[57 29 15]

Total iteration: 200
Imputation MAPE: 0.0871941
Imputation RMSE: 3.75304
Current rank:
[57 29 15]
Running time: 41 seconds


In [26]:
import scipy.io

tensor = scipy.io.loadmat('../../Guangzhou-data-set/tensor.mat')
dense_tensor = tensor['tensor']
random_tensor = scipy.io.loadmat('../../Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

missing_rate = 0.4

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

dense_tensor = np.transpose(dense_tensor, [0, 2, 1])
sparse_tensor = np.transpose(sparse_tensor, [0, 2, 1])

del tensor, random_tensor,binary_tensor

In [27]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.05
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 19s
MAPE: 0.106254
RMSE: 4.42741
Current rank:
[29 15  8]

Iteration: 200, Time cost: 53s
MAPE: 0.0895765
RMSE: 3.92271
Current rank:
[101  51  26]

Total iteration: 200
Imputation MAPE: 0.0895765
Imputation RMSE: 3.92271
Current rank:
[101  51  26]
Running time: 73 seconds


In [28]:
import scipy.io

tensor = scipy.io.loadmat('../../Guangzhou-data-set/tensor.mat')
dense_tensor = tensor['tensor']
random_tensor = scipy.io.loadmat('../../Guangzhou-data-set/random_tensor.mat')
random_tensor = random_tensor['random_tensor']

missing_rate = 0.6

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

dense_tensor = np.transpose(dense_tensor, [0, 2, 1])
sparse_tensor = np.transpose(sparse_tensor, [0, 2, 1])

del tensor, random_tensor,binary_tensor

In [29]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.05
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 17s
MAPE: 0.106097
RMSE: 4.41512
Current rank:
[13  7  4]

Iteration: 200, Time cost: 25s
MAPE: 0.0916796
RMSE: 3.94248
Current rank:
[49 25 13]

Total iteration: 200
Imputation MAPE: 0.0916796
Imputation RMSE: 3.94248
Current rank:
[49 25 13]
Running time: 43 seconds


We generate **non-random missing (NM)** values on Guangzhou traffic speed data set. Then, we conduct the imputation experiment.

In [42]:
import scipy.io

tensor = scipy.io.loadmat('../../Guangzhou-data-set/tensor.mat')
dense_tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../../Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']

missing_rate = 0.2

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

dense_tensor = np.transpose(dense_tensor, [0, 2, 1])
sparse_tensor = np.transpose(sparse_tensor, [0, 2, 1])

del tensor, random_matrix, binary_tensor

In [43]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.01
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 12s
MAPE: 0.109597
RMSE: 4.50428
Current rank:
[9 5 3]

Iteration: 200, Time cost: 13s
MAPE: 0.103317
RMSE: 4.32208
Current rank:
[13  7  4]

Total iteration: 200
Imputation MAPE: 0.103317
Imputation RMSE: 4.32208
Current rank:
[13  7  4]
Running time: 26 seconds


In [44]:
import scipy.io

tensor = scipy.io.loadmat('../../Guangzhou-data-set/tensor.mat')
dense_tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../../Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']

missing_rate = 0.4

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

dense_tensor = np.transpose(dense_tensor, [0, 2, 1])
sparse_tensor = np.transpose(sparse_tensor, [0, 2, 1])

del tensor, random_matrix, binary_tensor

In [45]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.01
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 18s
MAPE: 0.11712
RMSE: 4.83729
Current rank:
[17  9  5]

Iteration: 200, Time cost: 27s
MAPE: 0.101247
RMSE: 4.50245
Current rank:
[37 19 10]

Total iteration: 200
Imputation MAPE: 0.101247
Imputation RMSE: 4.50245
Current rank:
[37 19 10]
Running time: 46 seconds


In [46]:
import scipy.io

tensor = scipy.io.loadmat('../../Guangzhou-data-set/tensor.mat')
dense_tensor = tensor['tensor']
random_matrix = scipy.io.loadmat('../../Guangzhou-data-set/random_matrix.mat')
random_matrix = random_matrix['random_matrix']

missing_rate = 0.6

### Non-random missing (NM) scenario:
binary_tensor = np.zeros(dense_tensor.shape)
for i1 in range(dense_tensor.shape[0]):
    for i2 in range(dense_tensor.shape[1]):
        binary_tensor[i1, i2, :] = np.round(random_matrix[i1, i2] + 0.5 - missing_rate)
sparse_tensor = np.multiply(dense_tensor, binary_tensor)

dense_tensor = np.transpose(dense_tensor, [0, 2, 1])
sparse_tensor = np.transpose(sparse_tensor, [0, 2, 1])

del tensor, random_matrix, binary_tensor

In [47]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.01
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 16s
MAPE: 0.109879
RMSE: 4.56395
Current rank:
[13  7  4]

Iteration: 200, Time cost: 20s
MAPE: 0.103741
RMSE: 4.49001
Current rank:
[29 15  8]

Total iteration: 200
Imputation MAPE: 0.103741
Imputation RMSE: 4.49001
Current rank:
[29 15  8]
Running time: 37 seconds


### PeMS data

In [54]:
np.random.seed(122)

In [55]:
dense_mat = np.load('../../PeMS-data-set/pems.npy')
random_tensor = np.load('../../PeMS-data-set/random_tensor.npy')

missing_rate = 0.2

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, ten2mat(binary_tensor, 0))

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_tensor, binary_tensor

In [56]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.03
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 40s
MAPE: 0.0907084
RMSE: 6.1099
Current rank:
[21 29  5]

Iteration: 200, Time cost: 88s
MAPE: 0.0758795
RMSE: 5.21915
Current rank:
[71 99 15]

Total iteration: 200
Imputation MAPE: 0.0758795
Imputation RMSE: 5.21915
Current rank:
[71 99 15]
Running time: 129 seconds


In [57]:
dense_mat = np.load('../../PeMS-data-set/pems.npy')
random_tensor = np.load('../../PeMS-data-set/random_tensor.npy')

missing_rate = 0.4

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, ten2mat(binary_tensor, 0))

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_tensor, binary_tensor

In [58]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.03
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 28s
MAPE: 0.0974386
RMSE: 6.53541
Current rank:
[11 15  3]

Iteration: 200, Time cost: 66s
MAPE: 0.0791433
RMSE: 5.42467
Current rank:
[61 85 13]

Total iteration: 200
Imputation MAPE: 0.0791433
Imputation RMSE: 5.42467
Current rank:
[61 85 13]
Running time: 95 seconds


In [59]:
dense_mat = np.load('../../PeMS-data-set/pems.npy')
random_tensor = np.load('../../PeMS-data-set/random_tensor.npy')

missing_rate = 0.6

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, ten2mat(binary_tensor, 0))

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_tensor, binary_tensor

In [60]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.03
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 39s
MAPE: 0.0948544
RMSE: 6.33061
Current rank:
[21 29  5]

Iteration: 200, Time cost: 67s
MAPE: 0.0824886
RMSE: 5.66474
Current rank:
[51 71 11]

Total iteration: 200
Imputation MAPE: 0.0824886
Imputation RMSE: 5.66474
Current rank:
[51 71 11]
Running time: 107 seconds


In [61]:
dense_mat = np.load('../../PeMS-data-set/pems.npy')
random_matrix = np.load('../../PeMS-data-set/random_matrix.npy')

missing_rate = 0.2

### Nonrandom missing (NM) scenario:
binary_tensor = np.zeros((dense_mat.shape[0], 288, 44))
for i1 in range(dense_mat.shape[0]):
    for i2 in range(44):
        binary_tensor[i1,:,i2] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = ten2mat(binary_tensor, 0)
sparse_mat = np.multiply(dense_mat, binary_mat)

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_matrix, binary_tensor

In [62]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.03
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 39s
MAPE: 0.101119
RMSE: 7.89114
Current rank:
[21 29  5]

Iteration: 200, Time cost: 77s
MAPE: 0.0943265
RMSE: 6.66562
Current rank:
[61 85 13]

Total iteration: 200
Imputation MAPE: 0.0943265
Imputation RMSE: 6.66562
Current rank:
[61 85 13]
Running time: 117 seconds


In [63]:
dense_mat = np.load('../../PeMS-data-set/pems.npy')
random_matrix = np.load('../../PeMS-data-set/random_matrix.npy')

missing_rate = 0.4

### Nonrandom missing (NM) scenario:
binary_tensor = np.zeros((dense_mat.shape[0], 288, 44))
for i1 in range(dense_mat.shape[0]):
    for i2 in range(44):
        binary_tensor[i1,:,i2] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = ten2mat(binary_tensor, 0)
sparse_mat = np.multiply(dense_mat, binary_mat)

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_matrix, binary_tensor

In [64]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.03
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 37s
MAPE: 0.1001
RMSE: 6.65064
Current rank:
[21 29  5]

Iteration: 200, Time cost: 73s
MAPE: 0.0983528
RMSE: 7.09926
Current rank:
[56 78 12]

Total iteration: 200
Imputation MAPE: 0.0983528
Imputation RMSE: 7.09926
Current rank:
[56 78 12]
Running time: 111 seconds


In [65]:
dense_mat = np.load('../../PeMS-data-set/pems.npy')
random_matrix = np.load('../../PeMS-data-set/random_matrix.npy')

missing_rate = 0.6

### Nonrandom missing (NM) scenario:
binary_tensor = np.zeros((dense_mat.shape[0], 288, 44))
for i1 in range(dense_mat.shape[0]):
    for i2 in range(44):
        binary_tensor[i1,:,i2] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = ten2mat(binary_tensor, 0)
sparse_mat = np.multiply(dense_mat, binary_mat)

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_matrix, binary_tensor

In [66]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.03
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 46s
MAPE: 0.154229
RMSE: 10.1012
Current rank:
[36 50  8]

Iteration: 200, Time cost: 133s
MAPE: 0.135705
RMSE: 9.64099
Current rank:
[ 91 127  19]

Total iteration: 200
Imputation MAPE: 0.135705
Imputation RMSE: 9.64099
Current rank:
[ 91 127  19]
Running time: 179 seconds


### Electricity data

- **Random Missing (RM)**:

In [67]:
np.random.seed(113)

In [68]:
dense_mat = np.load('../../Electricity-data-set/electricity35.npy')
random_tensor = np.load('../../Electricity-data-set/random_tensor.npy')

missing_rate = 0.2

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, ten2mat(binary_tensor, 0))

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_tensor, binary_tensor

In [69]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.008
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 2s
MAPE: 0.2105
RMSE: 708.269
Current rank:
[40  4  4]

Iteration: 200, Time cost: 4s
MAPE: 0.174503
RMSE: 679.111
Current rank:
[53  5  5]

Total iteration: 200
Imputation MAPE: 0.174503
Imputation RMSE: 679.111
Current rank:
[53  5  5]
Running time: 6 seconds


In [70]:
dense_mat = np.load('../../Electricity-data-set/electricity35.npy')
random_tensor = np.load('../../Electricity-data-set/random_tensor.npy')

missing_rate = 0.4

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, ten2mat(binary_tensor, 0))

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_tensor, binary_tensor

In [71]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.008
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 2s
MAPE: 0.185794
RMSE: 851.604
Current rank:
[53  5  5]

Iteration: 200, Time cost: 4s
MAPE: 0.171313
RMSE: 754.138
Current rank:
[53  5  5]

Total iteration: 200
Imputation MAPE: 0.171313
Imputation RMSE: 754.138
Current rank:
[53  5  5]
Running time: 7 seconds


In [72]:
dense_mat = np.load('../../Electricity-data-set/electricity35.npy')
random_tensor = np.load('../../Electricity-data-set/random_tensor.npy')

missing_rate = 0.6

### Random missing (RM) scenario:
binary_tensor = np.round(random_tensor + 0.5 - missing_rate)
sparse_mat = np.multiply(dense_mat, ten2mat(binary_tensor, 0))

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_tensor, binary_tensor

In [73]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.008
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 2s
MAPE: 0.218151
RMSE: 835.601
Current rank:
[27  3  3]

Iteration: 200, Time cost: 3s
MAPE: 0.199306
RMSE: 835.474
Current rank:
[40  4  4]

Total iteration: 200
Imputation MAPE: 0.199306
Imputation RMSE: 835.474
Current rank:
[40  4  4]
Running time: 6 seconds


- **Nonrandom Missing (NM)**:

In [74]:
dense_mat = np.load('../../Electricity-data-set/electricity35.npy')
random_matrix = np.load('../../Electricity-data-set/random_matrix.npy')

missing_rate = 0.2

### Nonrandom missing (NM) scenario:
binary_tensor = np.zeros((dense_mat.shape[0], 24, 35))
for i1 in range(dense_mat.shape[0]):
    for i2 in range(35):
        binary_tensor[i1,:,i2] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = ten2mat(binary_tensor, 0)
sparse_mat = np.multiply(dense_mat, binary_mat)

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_matrix, binary_tensor

In [75]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.008
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 2s
MAPE: 0.287032
RMSE: 857.318
Current rank:
[27  3  3]

Iteration: 200, Time cost: 3s
MAPE: 0.233761
RMSE: 3746.38
Current rank:
[53  5  5]

Total iteration: 200
Imputation MAPE: 0.233761
Imputation RMSE: 3746.38
Current rank:
[53  5  5]
Running time: 5 seconds


In [76]:
dense_mat = np.load('../../Electricity-data-set/electricity35.npy')
random_matrix = np.load('../../Electricity-data-set/random_matrix.npy')

missing_rate = 0.4

### Nonrandom missing (NM) scenario:
binary_tensor = np.zeros((dense_mat.shape[0], 24, 35))
for i1 in range(dense_mat.shape[0]):
    for i2 in range(35):
        binary_tensor[i1,:,i2] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = ten2mat(binary_tensor, 0)
sparse_mat = np.multiply(dense_mat, binary_mat)

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_matrix, binary_tensor

In [77]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.008
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 1s
MAPE: 0.257709
RMSE: 928.84
Current rank:
[14  2  2]

Iteration: 200, Time cost: 2s
MAPE: 0.23805
RMSE: 1005.09
Current rank:
[27  3  3]

Total iteration: 200
Imputation MAPE: 0.23805
Imputation RMSE: 1005.09
Current rank:
[27  3  3]
Running time: 4 seconds


In [78]:
dense_mat = np.load('../../Electricity-data-set/electricity35.npy')
random_matrix = np.load('../../Electricity-data-set/random_matrix.npy')

missing_rate = 0.6

### Nonrandom missing (NM) scenario:
binary_tensor = np.zeros((dense_mat.shape[0], 24, 35))
for i1 in range(dense_mat.shape[0]):
    for i2 in range(35):
        binary_tensor[i1,:,i2] = np.round(random_matrix[i1,i2] + 0.5 - missing_rate)
binary_mat = ten2mat(binary_tensor, 0)
sparse_mat = np.multiply(dense_mat, binary_mat)

sparse_tensor = mat2ten(sparse_mat, np.array(binary_tensor.shape), 0)
dense_tensor = mat2ten(dense_mat, np.array(binary_tensor.shape), 0)

del dense_mat, random_matrix, binary_tensor

In [79]:
import time
start = time.time()
r = np.array([1, 1, 1])
dim = sparse_tensor.shape
R_max = np.array(dim)
delta = np.array(np.round(dim / np.min(dim)), dtype = 'int')
epsilon = 0.008
lambda_l = 1
rho = 0.95
mu0 = 0.0001
mu_min = 0.0000001
maxiter = 200
tensor_hat = imputer(dense_tensor, sparse_tensor, r, R_max, lambda_l, rho, mu0, mu_min, delta, epsilon, maxiter)
end = time.time()
print('Running time: %d seconds'%(end - start))

Iteration: 100, Time cost: 1s
MAPE: 0.327773
RMSE: 1818.53
Current rank:
[1 1 1]

Iteration: 200, Time cost: 1s
MAPE: 0.324575
RMSE: 1710.44
Current rank:
[1 1 1]

Total iteration: 200
Imputation MAPE: 0.324575
Imputation RMSE: 1710.44
Current rank:
[1 1 1]
Running time: 3 seconds


### License

<div class="alert alert-block alert-danger">
<b>This work is released under the MIT license.</b>
</div>